#Expected Gradients and the Differences method


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm_notebook
from io import StringIO # creates an in-memory text stream that you can read from or write to, just like a file
from scipy import ndimage # performs image processing tasks such as filtering, interpolation, and measurements on images
from textwrap import wrap
from tqdm import tqdm
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image # performs image-related tasks like loading an image file, converting between different image formats
import io # works with streams like files, buffers, and other I/O operations
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.cuda.manual_seed(42)
from google.colab import drive

drive.mount('/content/drive')

adv_model = resnet18()
adv_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
adv_model.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
adv_model.fc = nn.Linear(512, 10)


adv_model.load_state_dict(torch.load("/content/drive/MyDrive/adv_cifar10_model.pth")["model_state_dict"])
adv_pruned_model = torch.load('/content/drive/MyDrive/adv_cifar10_model_pruned.pth')['model']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
adv_model.to(device)
adv_pruned_model.to(device)

# Switching to evaluation mode
adv_model.eval()
adv_pruned_model.eval()


## Visualization


In [12]:
# Visualizing expected gradients
def ConvertToGrayscale(attributions):
    return np.average(attributions, axis = 2)

def Polarity(attributions, polarity):
    if polarity == 'positive':
        return np.clip(attributions, 0, 1)
    elif polarity == 'negative':
        return np.clip(attributions, -1, 0)
    else:
        raise ValueError('Wrong value for polarity')

def ComputeThresholdByTopPercentage(attributions, percentage = 60, plot_distribution = True):
    if percentage < 0 or percentage > 100:
        raise ValueError('Percentage must be in range [0, 100]')

    if percentage == 100:
        return np.min(attributions)

    flat_attributions = attributions.flatten()
    sum_attributions = np.sum(flat_attributions)

    sorted_attributions = np.sort(np.abs(flat_attributions))[::-1]

    cum_sum = 100.0 * np.cumsum(sorted_attributions) / sum_attributions

    threshold_idx = np.where(cum_sum >= percentage)[0][0]
    threshold = sorted_attributions[threshold_idx]

    if plot_distribution:
        values_to_plot = np.where(cum_sum >= 95)[0][0]
        values_to_plot = max(values_to_plot, threshold_idx)
        plt.plot(np.arange(values_to_plot), sorted_attributions[:values_to_plot])
        plt.axvline(x = threshold_idx)
        plt.show()

    return threshold

def LinearTransform(attributions, clip_above_percentile = 99.9, clip_below_percentile = 70.0, low = 0.2, plot_distribution = False):
    if clip_above_percentile < 0 or clip_above_percentile > 100:
        raise ValueError('clip_above_percentile must be in range [0, 100]')

    if clip_below_percentile < 0 or clip_below_percentile > 100:
        raise ValueError('clip_below_percentile must be in range [0, 100]')

    if low > 1 or low < 0:
        raise ValueError('low must be in range [0, 1]')

    m = ComputeThresholdByTopPercentage(attributions, percentage = 100 - clip_above_percentile, plot_distribution = plot_distribution)
    e = ComputeThresholdByTopPercentage(attributions, percentage = 100 - clip_below_percentile, plot_distribution = plot_distribution)

    transformed = (1 - low) * (np.abs(attributions) - e) / (m - e) + low

    transformed *= np.sign(attributions)

    transformed *= (transformed >= low)

    transformed = np.clip(transformed, 0.0, 1.0)
    return transformed

def Binarize(attributions, threshold = 0.001):
    return attributions > threshold

def MorphologicalCleanUp(attributions, structure = np.ones((4, 4))):
    closed = ndimage.grey_closing(attributions, structure = structure)
    opened = ndimage.grey_opening(closed, structure = structure)
    return opened

def Outlines(attributions, percentage = 90, connected_component_structure = np.ones((3, 3)), plot_distributions = True):
    attributions = Binarize(attributions)
    attributions = ndimage.binary_fill_holes(attributions)

    connected_components, num_cc = ndimage.measurements.label(attributions, structure = connected_component_structure)
    sum_connected_components = np.sum(attributions[connected_components > 0])

    component_sums_and_structures = []

    for cc_idx in range(1, 1 + num_cc):
        cc_mask = connected_components == cc_idx
        component_sum = np.sum(attributions[cc_mask])
        component_sums_and_structures.append((component_sum, cc_mask))

    sorted_sums_and_masks = sorted(component_sums_and_structures, lambda x: x[0], reverse = True)
    sorted_sums = list(zip(*sorted_sums_and_masks))[0]
    cumulative_sums = np.cumsum(sorted_sums)
    cutoff_threshold = percentage * sum_connected_components / 100.0
    cutoff_idx = np.where(cumulative_sums >= cutoff_threshold)[0][0]

    if cutoff_idx > 2:
        cutoff_idx = 2

    borded_mask = np.zeros_like(attributions)
    for i in range(1 + cutoff_idx):
        borded_mask[sorted_sums_and_masks[i][1]] = 1

    if plot_distributions:
        plt.plot(np.arange(len(sorted_sums)), sorted_sums)
        plt.axvline(x = cutoff_idx)
        plt.show()

    eroded_mask = ndimage.binary_erosion(borded_mask, iterations = 1)
    borded_mask[eroded_mask] = 0

    return borded_mask

def Overlay(attributions, image):
  # Create a new figure for the overlay
    fig, ax = plt.subplots()
    image = np.expand_dims(ConvertToGrayscale(image), 2) * [1, 1, 1]
    plt.imshow(np.clip(0, 255, attributions).astype(np.uint8))
    plt.imshow(np.clip(0, 255, image).astype(np.uint8), alpha = 0.15)
    plt.tight_layout()
    plt.axis('off')
    buf = io.BytesIO()
    a = plt.savefig(buf, bbox_inches = 'tight')
    buf.seek(0)
    plt.close(fig)
    pil1 = PIL.Image.open(buf)
    return pil1

G = [0, 255, 38]
R = [255, 0, 0]

def Visualize(attributions, image, positive_channel=G, negative_channel=R, polarity='positive', clip_above_percentile=99.9, clip_below_percentile=0, morphological_cleanup=False, structure=np.ones((3, 3)), outlines=False, outlines_component_percentage=90, overlay=True, plot_distribution=False):
    if polarity == 'both':
        pos_attributions = Visualize(
            attributions, image, positive_channel=positive_channel,
            negative_channel=negative_channel, polarity='positive',
            clip_above_percentile=clip_above_percentile, clip_below_percentile=clip_below_percentile,
            morphological_cleanup=morphological_cleanup, outlines=outlines,
            outlines_component_percentage=outlines_component_percentage,
            overlay=False,
            plot_distribution=plot_distribution)

        neg_attributions = Visualize(
            attributions, image, positive_channel=positive_channel,
            negative_channel=negative_channel, polarity='negative',
            clip_above_percentile=clip_above_percentile, clip_below_percentile=clip_below_percentile,
            morphological_cleanup=morphological_cleanup, outlines=outlines,
            outlines_component_percentage=outlines_component_percentage,
            overlay=False,
            plot_distribution=plot_distribution)

        attributions = pos_attributions + neg_attributions

        if overlay:
            attributions = Overlay(attributions, image)

        return attributions

    elif polarity == 'positive':
        attributions = Polarity(attributions, polarity=polarity)
        channel = positive_channel
    elif polarity == 'negative':
        attributions = Polarity(attributions, polarity=polarity)
        attributions = np.abs(attributions)
        channel = negative_channel

    attributions = ConvertToGrayscale(attributions)

    attributions = LinearTransform(attributions,
                                   clip_above_percentile, clip_below_percentile,
                                   0.0,
                                   plot_distribution=plot_distribution)

    if morphological_cleanup:
        attributions = MorphologicalCleanUp(attributions, structure=structure)
    if outlines:
        attributions = Outlines(attributions,
                                percentage=outlines_component_percentage,
                                plot_distribution=plot_distribution)

    attributions = np.expand_dims(attributions, 2) * channel

    if overlay:
        attributions = Overlay(attributions, image)

    return attributions

##Expected Gradients

In [13]:
#Expected Gradients
class ExpectedGradients:
    def __init__(self, model, normalizer=lambda x: x):
        self.model = model
        self.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')
        self.normalizer = normalizer

    def ToTensor(self, inp):
        inp = np.array(inp)
        inp = np.transpose(inp, (-1, 0, 1))
        inp_tensor = torch.from_numpy(inp).float().unsqueeze(0).to(self.device)
        return inp_tensor

    def PredictionsGradients(self, inp, target_label_idx):
        inp = inp.clone().requires_grad_(True)
        outputs = self.model(inp)
        outputs = F.softmax(outputs, dim = -1)[:, target_label_idx]
        outputs.backward(torch.ones(outputs.shape).to(self.device))
        gradients = inp.grad.detach().clone()
        return gradients, outputs.detach().clone().cpu().numpy()

    def GenerateSaturatedBatches(self, inp, baseline, steps):
        alphas = torch.linspace(0.0, 1.0, steps+1).to(self.device)
        alphas = alphas[:, None, None, None]
        images = baseline + alphas * (inp - baseline)
        return images

    def ExpectedGrads(self, inp, target_label_idx, baseline, steps = 50, batch_size = 30):
        scaled_inputs = self.GenerateSaturatedBatches(inp, baseline, steps)
        gradients = []
        predictions = []
        for i in range(0, scaled_inputs.shape[0], batch_size):
            start = i
            end = min(start + batch_size, scaled_inputs.shape[0])
            batch = scaled_inputs[start:end]
            batch = self.normalizer(batch)
            gradient, pred = self.PredictionsGradients(batch, target_label_idx)
            predictions.append(pred)
            gradients.append(gradient)
        predictions = np.hstack(predictions)
        gradients = torch.cat(gradients, axis = 0)
        inp = self.normalizer(inp)
        baseline = self.normalizer(baseline)
        gradients = (gradients[:-1] + gradients[1:]) / 2.0
        integrated_grads = torch.mean(gradients, axis = 0)
        delta_X = (inp - baseline).to(self.device)
        integrated_grads = (delta_X * integrated_grads).squeeze(0).detach().cpu().numpy()
        integrated_grads = np.transpose(integrated_grads, (1, 2, 0))
        completeness = np.abs(np.sum(integrated_grads) - (predictions[-1] - predictions[0]))
        return integrated_grads, completeness, predictions

    def Visualization(self, grad, image, treshold = 0):
        return Visualize(grad, (image * 255).astype(np.uint8), clip_below_percentile = treshold)

    def GaussianNoise(self, inp, target_label_idx, steps, num_random_trials, batch_size = 30, tqdm_p = True):
        inp = self.ToTensor(inp)
        all_intgrads = []
        itr = range(num_random_trials)
        if tqdm_p == True:
            itr = tqdm(itr, unit="trial")
        completeness = []
        preds = []
        for i in itr:
            integrated_grad, cm, predictions = self.ExpectedGrads(inp, target_label_idx, baseline=torch.normal(0, 0.4, inp.shape).to(self.device), steps = steps, batch_size = batch_size)
            preds.append(predictions)
            all_intgrads.append(integrated_grad)
            completeness.append(cm)
            if tqdm_p:
                itr.set_postfix(completeness = np.average(completeness))
        exp_grads = np.average(np.array(all_intgrads), axis = 0)
        return exp_grads, preds

    def RandomBaseline(self, inp, target_label_idx, steps, num_random_trials, batch_size = 30, tqdm_p = True):
        inp = self.ToTensor(inp)
        all_intgrads = []
        itr = range(num_random_trials)
        if tqdm_p == True:
            itr = tqdm(itr, unit = "trial")
        completeness = []
        preds = []
        for i in itr:
            integrated_grad, cm, predictions = self.ExpectedGrads(inp, target_label_idx, baseline = torch.rand(inp.shape).to(self.device), steps = steps, batch_size = batch_size)
            preds.append(predictions)
            all_intgrads.append(integrated_grad)
            completeness.append(cm)
            if tqdm_p:
                itr.set_postfix(completeness=np.average(completeness))
        exp_grads = np.average(np.array(all_intgrads), axis = 0)
        return exp_grads, preds

In [ ]:
# Transformations, loading dataset and getting a batch for visualization
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
normalizer = transforms.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768))
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

test_loader = DataLoader(test_set, batch_size=200, shuffle=False)
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

images, labels =  next(iter(test_loader))

## Plots and the Differences Method

In [ ]:
def expected_gradients_compare(adv_model, adv_pruned_model, images, labels, classes, normalizer=lambda x: x):
    adv_expected_grad = ExpectedGradients(adv_model, normalizer = normalizer)
    adv_pruned_expected_grad = ExpectedGradients(adv_pruned_model, normalizer = normalizer)
    number_of_img = labels.shape[0]
    labels_text = ['\n'.join(wrap(classes[l.item()], 20)) for l in labels.cpu()]
    num_columns = np.ceil(number_of_img / 12).astype(int)
    fig = plt.figure(figsize = (15.6 * num_columns, 48))
    subfigs = fig.subfigures(nrows = 1, ncols = num_columns)
    k = 0
    for j, sub in enumerate(subfigs):
        sub2 = sub.subfigures(nrows = 8, ncols = 1)
        for i, subfig in enumerate(sub2):
            if number_of_img == k:
                break
            axs = subfig.subplots(nrows=1, ncols=4)
            if i == 0:
                axs[0].set_title("Original image", fontsize = 20)
                axs[1].set_title("Robust", fontsize = 20)
                axs[2].set_title("Robust pruned", fontsize = 20)
                axs[3].set_title("Differences", fontsize = 20)
            img = images[k:k+1].to(device)
            img = normalizer(img)
            adv_prediction = adv_model(img)
            adv_pruned_prediction = adv_pruned_model(img)
            image = images[k].cpu().permute(1, 2, 0).numpy()
            adv_grad, _ = adv_expected_grad.RandomBaseline(image, adv_prediction.argmax(dim=1).item(), steps=50, num_random_trials=100, batch_size=30)
            adv_pruned_grad, _ = adv_pruned_expected_grad.RandomBaseline(image, adv_pruned_prediction.argmax(dim=1).item(), steps=50, num_random_trials=100, batch_size=30)

            adv_pred = '\n'.join(wrap(classes[adv_prediction.argmax(dim=1).item()], 12))
            adv_pruned_pred = '\n'.join(wrap(classes[adv_pruned_prediction.argmax(dim=1).item()], 12))

            axs[0].set_xlabel('\n'.join(wrap(labels_text[k], 10)), fontsize=35)
            axs[0].imshow(image)

            axs[1].set_xlabel(f"{adv_pred}", fontsize=35)
            axs[1].imshow(adv_expected_grad.Visualization(adv_grad, image))

            axs[2].set_xlabel(f"{adv_pruned_pred}", fontsize=35)
            axs[2].imshow(adv_pruned_expected_grad.Visualization(adv_pruned_grad, image))

            # The Differences Method
            adv_grad = np.average(adv_grad, axis = 2)
            adv_pruned_grad = np.average(adv_pruned_grad, axis = 2)

            sum_adv_gradients = np.sum(adv_grad)
            sum_adv_pruned_gradients = np.sum(adv_pruned_grad)


            if(sum_adv_gradients > sum_adv_pruned_gradients):
                scalling_difference = sum_adv_gradients - sum_adv_pruned_gradients
                value_per_pixel = scalling_difference / 1024 # 32 pixels by 32 pixels

                for i in range(32):
                    for j in range(32):
                        adv_pruned_grad[i][j] += value_per_pixel



            if(sum_adv_pruned_gradients > sum_adv_gradients):
                scalling_difference = sum_adv_pruned_gradients - sum_adv_gradients
                value_per_pixel = scalling_difference / 1024 # 32 pixels by 32 pixels

                for i in range(32):
                    for j in range(32):
                        adv_grad[i][j] += value_per_pixel



            difference_image = torch.zeros(32, 32, 3, dtype=torch.uint8)
            for i in range(32):
                for j in range(32):
                    if((adv_pruned_grad[i][j] - adv_grad[i][j]) > 0):
                        difference_image[i][j] = torch.tensor([0, 255, 38], dtype=torch.uint8)
                    if((adv_pruned_grad[i][j] - adv_grad[i][j]) < 0):
                        difference_image[i][j] = torch.tensor([255, 0, 0], dtype=torch.uint8)
            difference_image_np = difference_image.numpy()


            axs[3].imshow(difference_image_np)

            for t in range(4):
                axs[t].set_xticklabels([])
                axs[t].set_yticklabels([])
                axs[t].spines['top'].set_visible(False)
                axs[t].spines['right'].set_visible(False)
                axs[t].spines['left'].set_visible(False)
                axs[t].spines['bottom'].set_visible(False)
            k += 1